In [1]:
# # Let's take a look at the dataset image
# import mmcv
# import matplotlib.pyplot as plt

# img = mmcv.imread('data/coco/Locount_ImagesTest/000005.jpg')
# plt.figure(figsize=(15, 10))
# plt.imshow(mmcv.bgr2rgb(img))
# plt.show()

In [2]:
from mmengine import Config
cfg = Config.fromfile('./configs/cascade_rcnn/cascade-rcnn_r50_fpn_1x_coco_locount.py')

In [3]:
cfg

Config (path: ./configs/cascade_rcnn/cascade-rcnn_r50_fpn_1x_coco_locount.py): {'model': {'type': 'CascadeRCNN', 'data_preprocessor': {'type': 'DetDataPreprocessor', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'bgr_to_rgb': True, 'pad_size_divisor': 32}, 'backbone': {'type': 'ResNet', 'depth': 50, 'num_stages': 4, 'out_indices': (0, 1, 2, 3), 'frozen_stages': 1, 'norm_cfg': {'type': 'BN', 'requires_grad': True}, 'norm_eval': True, 'style': 'pytorch', 'init_cfg': {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}}, 'neck': {'type': 'FPN', 'in_channels': [256, 512, 1024, 2048], 'out_channels': 256, 'num_outs': 5}, 'rpn_head': {'type': 'RPNHead', 'in_channels': 256, 'feat_channels': 256, 'anchor_generator': {'type': 'AnchorGenerator', 'scales': [8], 'ratios': [0.5, 1.0, 2.0], 'strides': [4, 8, 16, 32, 64]}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [1.0, 1.0, 1.0, 1.0]}, 'loss_cls': {'type': 'CrossE

In [4]:
from mmengine.runner import set_random_seed

# Modify dataset classes and color
cfg.metainfo = {
    'classes': ('Thermos bottle', 'Soymilk', 'Baby slippers', 'Makeup tools', 'Adult milk powder', 'Red wine', 'Toothpaste', 'Pot shovel', 'Rubber ball', 'Rise', 'Stool', 'Vinegar', 'Children Socks', 'Shampoo', 'Cake', 'Pen', 'Flour', 'Sesame paste', 'Dried meat', 'Baby diapers', 'Jacket', 'Herbal tea', 'Mouth wash', 'Care Kit', 'Baby Toys', 'Quick-frozen Wonton', 'Quick-frozen Tangyuan', 'Electric Hot pot', 'Electric steaming pan', 'Electric kettle', 'Sports cup', 'Quick-frozen dumplings', 'Can', 'Air conditioner', 'Instant noodles', 'Baby washing and nursing supplie', 'Socket', 'Potato chips', 'Disposable bag', 'Adult shoes', 'Football', 'Hair drier', 'Chopping block', 'Lingerie', 'Mug', 'Soup ladle', 'Children hats', 'Lotus root flour', 'Fish tofu', 'Electric iron', 'Ice cream', 'Bath lotion', 'Juicer', 'Tea beverage', 'Dinner plate', 'Notebook', 'Facial Cleanser', 'Biscuits', 'Chewing gum', 'Mixed congee', 'Facial mask', 'Bowl', 'Soybean Milk machine', 'Men underwear', 'Dairy', 'Hair gel', 'Tea', 'Storage bottle', 'Spoon', 'Baby milk powder', 'Sauce', 'Electric fan', 'Dried beans', 'Disposable cups', 'Draw bar box', 'Cooking wine', 'Pencil case', 'Carbonated drinks', 'Adult Diapers', 'Soy sauce', 'Desk lamp', 'Electromagnetic furnace', 'Badminton', 'Coffee', 'Air conditioning fan', 'Tampon', 'Children Toys', 'Children underwear', 'Knives', 'Pasta', 'Microwave Oven', 'Hair conditioner', 'Ginger Tea', 'Baby Furniture', 'Forks', 'Pie', 'Television', 'Baby tableware', 'Skate', 'Oats', 'Cotton swab', 'Storage box', 'Noodle', 'Razor', 'Refrigerator', 'Toothbrush', 'Chocolates', 'Adult socks', 'Knapsack', 'Sour Plum Soup', 'Cutter', 'Band aid', 'Rice cooker', 'Cocktail', 'Skin care set', 'Walnut powder', 'Liquor and Spirits', 'Emulsion', 'Soap', 'Hair dye', 'Fresh-keeping film', 'Baby handkerchiefs ', 'Children shoes', 'Washing machine', 'Chopsticks', 'Guozhen', 'Trousers', 'Adult hat', 'Trash', 'Bedding set', 'Basin', 'Electric frying pan', 'Food box', 'Dried fish', 'Baby carriage', 'Hot strips', 'Basketball', 'Comb', 'Coat hanger')
}

# Modify dataset type and path
cfg.data_root = 'data/coco'

cfg.train_dataloader.dataset.ann_file = 'annotations/annotation_train.json'
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.train_dataloader.dataset.data_prefix.img = 'Locount_ImagesTrain'
cfg.train_dataloader.dataset.metainfo = cfg.metainfo

cfg.val_dataloader.dataset.ann_file = 'annotations/annotation_test.json'
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_prefix.img = 'Locount_ImagesTest'
cfg.val_dataloader.dataset.metainfo = cfg.metainfo

cfg.test_dataloader = cfg.val_dataloader

# Modify metric config
cfg.val_evaluator.ann_file = cfg.data_root+'/'+'annotations/annotation_test.json'
cfg.test_evaluator = cfg.val_evaluator

# Modify num classes of the model in box head and mask head
# cfg.model.roi_head.bbox_head.num_classes = 140
# cfg.model.roi_head.mask_head.num_classes = 140

# We can still the pre-trained Mask RCNN model to obtain a higher performance
# cfg.load_from = 'checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'


# We can set the evaluation interval to reduce the evaluation times
cfg.train_cfg.val_interval = 3
# We can set the checkpoint saving interval to reduce the storage cost
cfg.default_hooks.checkpoint.interval = 3

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optim_wrapper.optimizer.lr = 0.02 / 8
cfg.default_hooks.logger.interval = 10


# Set seed thus the results are more reproducible
# cfg.seed = 0
set_random_seed(0, deterministic=False)

# We can also use tensorboard to log the training process
cfg.visualizer.vis_backends.append({"type":'TensorboardVisBackend'})

/opt/anaconda3/envs/openmmlab/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from mmengine.runner import Runner

# build the runner from config
runner = Runner.from_cfg(cfg)

07/16 21:04:35 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: darwin
    Python: 3.8.17 (default, Jul  5 2023, 16:18:40) [Clang 14.0.6 ]
    CUDA available: False
    numpy_random_seed: 209652396
    GCC: Apple clang version 14.0.3 (clang-1403.0.22.14.1)
    PyTorch: 1.13.1
    PyTorch compiling details: PyTorch built with:
  - GCC 4.2
  - C++ Version: 201402
  - clang 14.0.6
  - OpenMP 201811
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: NO AVX
  - Build settings: BLAS_INFO=open, BUILD_TYPE=Release, CXX_COMPILER=/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_9d63z49rj_/croot/pytorch_1681837279022/_build_env/bin/x86_64-apple-darwin13.4.0-clang++, CXX_FLAGS=-march=core2 -mtune=haswell -mssse3 -ftree-vectorize -fPIC -fPIE -fstack-protector-strong -O2 -pipe -stdlib=libc++  -fmessage-length=0 -isystem /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_9d63z49rj

/opt/anaconda3/envs/openmmlab/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/envs/openmmlab/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowIxEET_S2_S2_b
  Referenced from: <870081F6-12FD-3CEA-BC5C-30F4764F2A98> /opt/anaconda3/envs/openmmlab/lib/python3.8/site-packages/torchvision/image.so
  Expected in:     <F2FE5CF8-5B5B-3FAD-ADF8-C77D90F49FC9> /opt/anaconda3/envs/openmmlab/lib/python3.8/site-packages/torch/lib/libc10.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


07/16 21:04:36 - mmengine - INFO - Config:
model = dict(
    type='CascadeRCNN',
    data_preprocessor=dict(
        type='DetDataPreprocessor',
        mean=[
            123.675,
            116.28,
            103.53,
        ],
        std=[
            58.395,
            57.12,
            57.375,
        ],
        bgr_to_rgb=True,
        pad_size_divisor=32),
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(
            0,
            1,
            2,
            3,
        ),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[
            256,
            512,
            1024,
            2048,
        ],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channel

In [7]:
runner.train()

07/16 21:12:29 - mmengine - WARNING - init_weights of CascadeRCNN has been called more than once.
07/16 21:12:29 - mmengine - INFO - Checkpoints will be saved to /Users/nuthankumar/Data/Codes/Locount/Locount/raw/Locount/tutorial_exps.


KeyboardInterrupt: 

In [ ]:
%load_ext tensorboard